- https://docs.google.com/spreadsheets/d/1-hYJesNCMlCyANPOPeLijg1sCfv-6nREPtkHtTVEwd4/edit#gid=0
- https://observablehq.com/@carbonfact/diff

https://chat.openai.com/share/6a5e1c60-b9a2-42bb-9b23-39e61358b577

In [1]:
import pandas as pd
import random

random.seed(42)

# Function to generate a random cost based on the claim type and year
def generate_claim_cost(claim_type, year):
    if claim_type == 'Dentist':
        base_cost = 100
    elif claim_type == 'Psychiatrist':
        base_cost = 150
    elif claim_type == 'General Physician':
        base_cost = 80
    elif claim_type == 'Physiotherapy':
        base_cost = 120
    else:
        base_cost = 50

    # Adjust cost based on year
    if year == 2021:
        base_cost *= 1.2
    elif year == 2023:
        base_cost *= 1.5

    # Add some random variation
    cost = random.uniform(base_cost - 20, base_cost + 20)
    return round(cost, 2)

# Generating sample data
claim_types = ['Dentist', 'Psychiatrist', 'General Physician', 'Physiotherapy']
years = [2021, 2022, 2023]
people = ['John', 'Jane', 'Michael', 'Emily', 'William', 'Emma', 'Daniel', 'Olivia', 'Lucas', 'Ava']

data = []
for year in years:
    for person in people:
        num_claims = random.randint(1, 5)  # Random number of claims per person per year
        for _ in range(num_claims):
            claim_type = random.choice(claim_types)
            cost = generate_claim_cost(claim_type, year)
            date = pd.to_datetime(f"{random.randint(1, 12)}/{random.randint(1, 28)}/{year}", format='%m/%d/%Y')
            data.append([person, claim_type, date, year, cost])

# Create the DataFrame
columns = ['person', 'claim_type', 'date', 'year', 'amount']
claims_df = pd.DataFrame(data, columns=columns)

# Display the DataFrame
claims_df.head()

,person,claim_type,date,year,amount
0,John,Dentist,2021-04-08,2021,129.66
1,Jane,Dentist,2021-09-03,2021,127.07
2,Jane,Physiotherapy,2021-02-07,2021,125.27
3,Michael,Dentist,2021-12-21,2021,122.45
4,Michael,Physiotherapy,2021-10-09,2021,132.82


In [2]:
len(claims_df)

80

In [3]:
print(claims_df.head().to_markdown(index=False))

| person   | claim_type    | date                |   year |   amount |
|:---------|:--------------|:--------------------|-------:|---------:|
| John     | Dentist       | 2021-04-08 00:00:00 |   2021 |   129.66 |
| Jane     | Dentist       | 2021-09-03 00:00:00 |   2021 |   127.07 |
| Jane     | Physiotherapy | 2021-02-07 00:00:00 |   2021 |   125.27 |
| Michael  | Dentist       | 2021-12-21 00:00:00 |   2021 |   122.45 |
| Michael  | Physiotherapy | 2021-10-09 00:00:00 |   2021 |   132.82 |


## Sums

- Total cost
- Total footprint

In [4]:
sums = claims_df.groupby('year')['amount'].sum()
sums = pd.DataFrame({'sum': sums, 'diff': sums - sums.shift()})
print(sums.to_markdown())

|   year |     sum |    diff |
|-------:|--------:|--------:|
|   2021 | 3814.54 |  nan    |
|   2022 | 2890.29 | -924.25 |
|   2023 | 4178.03 | 1287.74 |


In [5]:
print(pd.DataFrame({
    'sum': (s := claims_df.groupby(['year', 'claim_type'])['amount'].sum()),
    'diff': (
        s - s.groupby('claim_type').shift()
    )
}).reset_index().to_markdown(index=False))

|   year | claim_type        |     sum |    diff |
|-------:|:------------------|--------:|--------:|
|   2021 | Dentist           | 1104.42 |  nan    |
|   2021 | General Physician |  594.44 |  nan    |
|   2021 | Physiotherapy     |  801.78 |  nan    |
|   2021 | Psychiatrist      | 1313.9  |  nan    |
|   2022 | Dentist           |  622.48 | -481.94 |
|   2022 | General Physician |  749.08 |  154.64 |
|   2022 | Physiotherapy     |  339.45 | -462.33 |
|   2022 | Psychiatrist      | 1179.28 | -134.62 |
|   2023 | Dentist           | 1440.99 |  818.51 |
|   2023 | General Physician |  826.18 |   77.1  |
|   2023 | Physiotherapy     | 1049.15 |  709.7  |
|   2023 | Psychiatrist      |  861.71 | -317.57 |


In [6]:
metric = 'amount'
period = 'year'
dimension = 'claim_type'

totals = (
    claims_df
    .groupby([period, dimension])
    [metric]
    .agg(['mean', 'count', 'sum'])
    .reset_index()
    .sort_values(period)
)

totals['mean_lag'] = totals.groupby(dimension)['mean'].shift(1)
totals['count_lag'] = totals.groupby(dimension)['count'].shift(1)
totals['inner'] = totals.eval('(mean - mean_lag) * count_lag')
totals['mix'] = totals.eval('(count - count_lag) * mean')
totals

,year,claim_type,mean,count,sum,mean_lag,count_lag,inner,mix
0,2021,Dentist,122.713333,9,1104.42,NaN,NaN,NaN,NaN
1,2021,General Physician,99.073333,6,594.44,NaN,NaN,NaN,NaN
2,2021,Physiotherapy,133.630000,6,801.78,NaN,NaN,NaN,NaN
3,2021,Psychiatrist,187.700000,7,1313.90,NaN,NaN,NaN,NaN
4,2022,Dentist,103.746667,6,622.48,122.713333,9.0,-170.700000,-311.240000
5,2022,General Physician,83.231111,9,749.08,99.073333,6.0,-95.053333,249.693333
6,2022,Physiotherapy,113.150000,3,339.45,133.630000,6.0,-122.880000,-339.450000
7,2022,Psychiatrist,147.410000,8,1179.28,187.700000,7.0,-282.030000,147.410000
8,2023,Dentist,160.110000,9,1440.99,103.746667,6.0,338.180000,480.330000
9,2023,General Physician,118.025714,7,826.18,83.231111,9.0,313.151429,-236.051429


In [7]:
print(totals.groupby('year').apply(lambda x: (x.inner + x.mix).sum()).to_markdown())

|   year |       0 |
|-------:|--------:|
|   2021 |    0    |
|   2022 | -924.25 |
|   2023 | 1287.74 |


In [87]:
import collections
import random
import names  # This library generates human names
import pandas as pd

random.seed(42)

# Function to generate a random cost based on the claim type and year
def generate_claim_cost(claim_type, year):
    if claim_type == 'Dentist':
        base_cost = 100
    elif claim_type == 'Psychiatrist':
        base_cost = 150

    # Adjust cost based on year
    if year == 2021:
        base_cost *= 1.2
    elif year == 2023:
        base_cost *= 1.5

    # Add some random variation
    cost = random.uniform(base_cost - 20, base_cost + 20)
    return round(cost, 2)

# Generating sample data
claim_types = ['Dentist', 'Psychiatrist']
years = [2021, 2022, 2023, 2024]
people = ['John', 'Jane', 'Michael', 'Emily', 'William']

data = []
for year in years:
    new_people = (
        [names.get_first_name() for _ in range(random.randint(1, 3))]
        if year > 2021
        else []
    )
    existing_people = [person for person in people if random.random() > 0.3]
    people_this_year = existing_people + new_people
    people.extend(new_people)

    for person in people_this_year:
        num_claims = random.randint(1, 5)  # Random number of claims per existing customer per year
        for _ in range(num_claims):
            claim_type = random.choice(claim_types)
            cost = generate_claim_cost(claim_type, year)
            date = pd.to_datetime(f"{random.randint(1, 12)}/{random.randint(1, 28)}/{year}", format='%m/%d/%Y')
            data.append([person, claim_type, date, year, cost])

# Create the DataFrame
columns = ['person', 'claim_type', 'date', 'year', 'amount']
claims_df = pd.DataFrame(data, columns=columns)

# Indicate whether people are existing, new, or returning
years_seen = collections.defaultdict(set)
statuses = []
for claim in claims_df.to_dict(orient='records'):
    years_seen[claim['person']].add(claim['year'])
    if claim['year'] - 1 in years_seen[claim['person']]:
        statuses.append('EXISTING')
    elif any(year < claim['year'] for year in years_seen[claim['person']]):
        statuses.append('RETURNING')
    elif not {year for year in years_seen[claim['person']] if year != claim['year']}:
        statuses.append('NEW')

claims_df['status'] = statuses

print(claims_df.sample(5).to_markdown(index=False))

| person   | claim_type   | date                |   year |   amount | status   |
|:---------|:-------------|:--------------------|-------:|---------:|:---------|
| Emily    | Psychiatrist | 2024-10-16 00:00:00 |   2024 |   132.29 | EXISTING |
| Deborah  | Dentist      | 2023-01-28 00:00:00 |   2023 |   139.16 | NEW      |
| Emily    | Psychiatrist | 2023-01-24 00:00:00 |   2023 |   211.33 | NEW      |
| Keith    | Dentist      | 2022-01-08 00:00:00 |   2022 |   107.38 | NEW      |
| Jane     | Dentist      | 2023-03-26 00:00:00 |   2023 |   136.11 | NEW      |


In [10]:
print(totals.groupby('year').apply(lambda x: (x.inner + x.mix).sum()).to_markdown())

|   year |        0 |
|-------:|---------:|
|   2021 |  1312.28 |
|   2022 |  -636.22 |
|   2023 |  4515.25 |
|   2024 | -1223.09 |


In [11]:
sums = claims_df.groupby('year')['amount'].sum()
sums = pd.DataFrame({'sum': sums, 'diff': sums - sums.shift()})
print(sums.to_markdown())

|   year |     sum |     diff |
|-------:|--------:|---------:|
|   2021 | 1312.28 |   nan    |
|   2022 |  676.06 |  -636.22 |
|   2023 | 5191.31 |  4515.25 |
|   2024 | 1966.73 | -3224.58 |


## Ratios

- Cost by claim
- Cost by user
- Footprint by product
- Footprint by gram

In [93]:
averages = claims_df.groupby('year')['amount'].mean()
averages = pd.DataFrame({'average': averages, 'diff': averages - averages.shift()})
print(averages.to_markdown())

|   year |   average |     diff |
|-------:|----------:|---------:|
|   2021 |   145.809 | nan      |
|   2022 |   112.677 | -33.1322 |
|   2023 |   173.044 |  60.367  |
|   2024 |   122.921 | -50.123  |


In [94]:
metric = 'amount'
period = 'year'
dimension = 'claim_type'

decomp = (
    claims_df
    .groupby([period, dimension], dropna=True)
    [metric].agg(['sum', 'count'])
    .reset_index()
    .sort_values(period)
)
decomp.head()

,year,claim_type,sum,count
0,2021,Dentist,614.36,5
1,2021,Psychiatrist,697.92,4
2,2022,Dentist,393.50,4
3,2022,Psychiatrist,282.56,2
4,2023,Dentist,2967.30,20


In [95]:
decomp['mean'] = decomp.eval('sum / count')
decomp['share'] = decomp['count'] / decomp.groupby('year')['count'].transform('sum')
decomp['global_mean'] = (
    decomp.groupby('year')['sum'].transform('sum') /
    decomp.groupby('year')['count'].transform('sum')
)
decomp['mean_lag'] = decomp.groupby(dimension)['mean'].shift(1)
decomp['share_lag'] = decomp.groupby(dimension)['share'].shift(1)
decomp['global_mean_lag'] = decomp.groupby(dimension)['global_mean'].shift(1)
decomp['inner'] = decomp.eval('share * (mean - mean_lag)')
decomp['mix'] = decomp.eval('(share - share_lag) * (mean_lag - global_mean_lag)')
decomp

,year,claim_type,sum,count,mean,share,global_mean,mean_lag,share_lag,global_mean_lag,inner,mix
0,2021,Dentist,614.36,5,122.87200,0.555556,145.808889,NaN,NaN,NaN,NaN,NaN
1,2021,Psychiatrist,697.92,4,174.48000,0.444444,145.808889,NaN,NaN,NaN,NaN,NaN
2,2022,Dentist,393.50,4,98.37500,0.666667,112.676667,122.872,0.555556,145.808889,-16.331333,-2.548543
3,2022,Psychiatrist,282.56,2,141.28000,0.333333,112.676667,174.480,0.444444,145.808889,-11.066667,-3.185679
4,2023,Dentist,2967.30,20,148.36500,0.666667,173.043667,98.375,0.666667,112.676667,33.326667,-0.000000
5,2023,Psychiatrist,2224.01,10,222.40100,0.333333,173.043667,141.280,0.333333,112.676667,27.040333,0.000000
6,2024,Dentist,781.46,8,97.68250,0.500000,122.920625,148.365,0.666667,173.043667,-25.341250,4.113111
7,2024,Psychiatrist,1185.27,8,148.15875,0.500000,122.920625,222.401,0.333333,173.043667,-37.121125,8.226222


In [96]:
decomp.groupby('year').apply(lambda x: (x.inner + x.mix).sum()).sum()

-22.888263888888893

In [97]:
print(decomp.groupby('year').apply(lambda x: (x.inner + x.mix).sum()).to_markdown())

|   year |        0 |
|-------:|---------:|
|   2021 |   0      |
|   2022 | -33.1322 |
|   2023 |  60.367  |
|   2024 | -50.123  |


### DuckDB

In [94]:
import duckdb

duckdb.sql("CREATE TABLE claims AS SELECT * FROM claims_df")

CatalogException: Catalog Error: Table with name "claims" already exists!